In [ ]:
! pip install cmake
! pip install dlib

! pip install face_recognition mysql-connector-python opencv-python numpy scikit-learn pillow

: 

In [ ]:
import os
import face_recognition
import numpy as np
from sklearn import neighbors
import pickle
from PIL import Image
import cv2
import mysql.connector
import requests
from pathlib import Path

# Function to download dataset from MySQL
def download_dataset_from_mysql():
    # Create the dataset directory if it doesn't exist
    dataset_path = "dataset"
    os.makedirs(dataset_path, exist_ok=True)
    
    try:
        conn = mysql.connector.connect(
            host="103.245.39.59",
            user="rivo",
            password="@Rivopelu123",
            database="face_db"
        )
        cursor = conn.cursor(dictionary=True)

        query = """
            SELECT account.name AS student_name, student_images.image_url
            FROM student_images
            JOIN account ON student_images.account_id = account.id
            WHERE account.role = 'STUDENT' AND account.active = 1;
        """
        cursor.execute(query)
        rows = cursor.fetchall()

        for row in rows:
            name = row['student_name'].replace(" ", "_")
            image_url = row['image_url']

            student_folder = os.path.join(dataset_path, name)
            os.makedirs(student_folder, exist_ok=True)

            img_count = len(os.listdir(student_folder)) + 1
            img_path = os.path.join(student_folder, f"{img_count}.jpg")

            try:
                img_data = requests.get(image_url).content
                with open(img_path, 'wb') as f:
                    f.write(img_data)
                print(f"✅ Downloaded: {img_path}")
            except Exception as e:
                print(f"❌ Failed to download {image_url}: {e}")

        cursor.close()
        conn.close()
        print("Database connection closed successfully.")
        
    except mysql.connector.Error as err:
        print(f"MySQL Error: {err}")

# Function to train the KNN model
def train_knn_model(dataset_dir, model_save_path="trained_knn_model.clf", n_neighbors=2):
    X = []
    y = []
    
    # Check if dataset directory exists
    if not os.path.exists(dataset_dir):
        print(f"❌ Dataset directory {dataset_dir} not found!")
        return

    for student_dir in os.listdir(dataset_dir):
        student_path = os.path.join(dataset_dir, student_dir)

        if not os.path.isdir(student_path):
            continue

        for img_name in os.listdir(student_path):
            img_path = os.path.join(student_path, img_name)
            
            try:
                # Load image
                image = face_recognition.load_image_file(img_path)
                
                # Use HOG for face detection (works better on M1)
                face_bounding_boxes = face_recognition.face_locations(image, model='hog')

                if len(face_bounding_boxes) != 1:
                    print(f"❌ Skipped {img_path}: found {len(face_bounding_boxes)} faces")
                    continue

                # Get face encoding
                encoding = face_recognition.face_encodings(image, known_face_locations=face_bounding_boxes)[0]
                X.append(encoding)
                y.append(student_dir)
                print(f"✅ Processed: {img_path}")
            except Exception as e:
                print(f"❌ Error processing {img_path}: {e}")

    if len(X) == 0:
        print("❌ No faces found in any images. Cannot train model.")
        return
    
    print(f"Training model with {len(X)} face encodings...")
    
    # Train KNN classifier
    knn_clf = neighbors.KNeighborsClassifier(n_neighbors=n_neighbors, algorithm='ball_tree', weights='distance')
    knn_clf.fit(X, y)

    # Save the model
    with open(model_save_path, 'wb') as f:
        pickle.dump(knn_clf, f)

    print(f"✅ Model saved as {model_save_path} with {len(X)} face encodings from {len(set(y))} people")
    return knn_clf

# Function to recognize faces using webcam
def recognize_faces_webcam(model_path="trained_knn_model.clf", tolerance=0.6):
    # Load the trained KNN model
    if not os.path.exists(model_path):
        print(f"❌ Model file {model_path} not found! Please train the model first.")
        return
        
    with open(model_path, 'rb') as f:
        knn_clf = pickle.load(f)
    
    # Initialize webcam
    video_capture = cv2.VideoCapture(0)
    
    if not video_capture.isOpened():
        print("❌ Error: Could not open webcam.")
        return
    
    print("✅ Face recognition started. Press 'q' to quit.")
    
    while True:
        # Grab a single frame of video
        ret, frame = video_capture.read()
        if not ret:
            print("❌ Failed to grab frame from webcam")
            break
            
        # Convert from BGR (OpenCV format) to RGB (face_recognition format)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Find all face locations and encodings
        face_locations = face_recognition.face_locations(rgb_frame, model="hog")
        
        if face_locations:
            face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
            
            # Find matches for each face
            for face_encoding, face_location in zip(face_encodings, face_locations):
                # Use the KNN model to find the closest match
                closest_distances = knn_clf.kneighbors([face_encoding], n_neighbors=1)
                match = closest_distances[0][0][0] <= tolerance
                
                if match:
                    name = knn_clf.predict([face_encoding])[0]
                else:
                    name = "Unknown"
                
                # Draw box and name
                top, right, bottom, left = face_location
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
                font = cv2.FONT_HERSHEY_DUPLEX
                cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.8, (255, 255, 255), 1)
        
        # Display the resulting image
        cv2.imshow('Face Recognition', frame)
        
        # Hit 'q' on the keyboard to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release handle to the webcam
    video_capture.release()
    cv2.destroyAllWindows()

# Main execution
if __name__ == "__main__":
    print("Face Recognition System")
    print("=======================")
    
    while True:
        print("\nOptions:")
        print("1. Download dataset from MySQL")
        print("2. Train recognition model")
        print("3. Start face recognition using webcam")
        print("4. Exit")
        
        choice = input("\nEnter your choice (1-4): ")
        
        if choice == '1':
            download_dataset_from_mysql()
        elif choice == '2':
            train_knn_model("dataset")
        elif choice == '3':
            recognize_faces_webcam()
        elif choice == '4':
            print("Exiting program...")
            break
        else:
            print("Invalid choice. Please try again.")